## Imports

In [8]:
# System functionality
import sys
sys.path.append('..')

import glob
import os
import numpy as np 
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager
import seaborn as sns
import graphviz
import math as math
import json
import cv2
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from sympy import *
from skimage import data, io, filters
from numpy import linspace
from numpy.core.umath import pi
from matplotlib.ticker import FuncFormatter
from matplotlib.font_manager import FontProperties
from emoji_extractor.extract import Extractor
from emojipedia import Emojipedia
from skimage import io
from sklearn.preprocessing import Normalizer
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics.pairwise import chi2_kernel
#from utils.plotter import plot_SVM_DecisionBoundary
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, log_loss
from pandas.api.types import CategoricalDtype
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

## Load csvs

In [83]:
path_counts_ran = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/random_per_user_emoji_counts.csv'
path_users_ran = '/home/stelios/Desktop/Honours Project/Samples/user_data/ran.csv'
counts_df_ran = pd.read_csv(path_counts,index_col=None, header=0, low_memory = False)
users_df_ran = pd.read_csv(path_users,index_col=None, header=0, low_memory = False)

path_counts_lon = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/london_per_user_emoji_counts.csv'
path_users_lon = '/home/stelios/Desktop/Honours Project/Samples/user_data/lon.csv'
counts_df_lon = pd.read_csv(path_counts_lon,index_col=None, header=0, low_memory = False)
users_df_lon = pd.read_csv(path_users_lon,index_col=None, header=0, low_memory = False)

path_counts_joh = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/joh_per_user_emoji_counts.csv'
path_users_joh = '/home/stelios/Desktop/Honours Project/Samples/user_data/joh.csv'
counts_df_joh = pd.read_csv(path_counts_joh,index_col=None, header=0, low_memory = False)
users_df_joh = pd.read_csv(path_users_joh,index_col=None, header=0, low_memory = False)

#path_counts_nyc = '/home/stelios/Desktop/Honours Project/Samples/per_user_emoji_counts/nyc_per_user_emoji_counts.csv'
#path_users_nyc = '/home/stelios/Desktop/Honours Project/Samples/user_data/nyc.csv'
#counts_df_nyc = pd.read_csv(path_counts_nyc,index_col=None, header=0, low_memory = False)
#users_df_nyc = pd.read_csv(path_users_nyc,index_col=None, header=0, low_memory = False)

In [87]:
# Append counts from all Locations
counts = counts_df_ran.append(counts_df_lon, sort=False,ignore_index=True).append(counts_df_joh, sort=False,ignore_index=True)
print(counts.shape)

# Append users from all Locations
users = users_df_ran.append(users_df_lon, sort=False,ignore_index=True).append(users_df_joh, sort=False,ignore_index=True)
print(users.shape)

# Fill NaN with 0.0
counts = counts.fillna(0.0)

(30000, 2541)
(30000, 28)


In [91]:
# Preview counts
counts.describe()

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧙🏾‍♂️,🧙🏾,⛹🏼‍♀️,🏂🏾,🧜🏾‍♂️,🤸🏽,🦶🏾,🏋🏾,🇧🇹,🇳🇨
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.00000,30000.000000,30000.000000,30000.0000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000
mean,0.000033,0.000033,0.000067,0.000267,0.000200,0.00010,0.000267,0.000133,0.0001,0.000033,...,0.000133,0.000167,0.000167,0.000167,0.000267,0.000500,0.000633,0.000900,0.000033,0.000033
std,0.005774,0.005774,0.008165,0.031622,0.019999,0.01291,0.018256,0.011546,0.0100,0.005774,...,0.016330,0.028868,0.028868,0.028868,0.021601,0.056272,0.109697,0.083063,0.005774,0.005774
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.0000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,5.000000,3.000000,2.00000,2.000000,1.000000,1.0000,1.000000,...,2.000000,5.000000,5.000000,5.000000,2.000000,9.000000,19.000000,12.000000,1.000000,1.000000


## % Feuture representation

In [92]:
# Feature % representation
for index, row in counts.iterrows():
    counts.loc[index] = counts.loc[index]/counts.loc[index].sum()
counts.describe()

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧙🏾‍♂️,🧙🏾,⛹🏼‍♀️,🏂🏾,🧜🏾‍♂️,🤸🏽,🦶🏾,🏋🏾,🇧🇹,🇳🇨
count,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,...,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,2.041900e+04,20419.000000,2.041900e+04,20419.000000,2.041900e+04,2.041900e+04
mean,3.781776e-08,2.365893e-07,4.841387e-08,8.318272e-07,4.717043e-07,2.217353e-07,4.497027e-07,9.129696e-07,8.110759e-07,5.761646e-07,...,2.809206e-07,7.309551e-07,9.282410e-08,3.139359e-07,4.512796e-07,0.000001,4.350191e-07,0.000003,6.529866e-07,2.147982e-07
std,5.403971e-06,3.380745e-05,5.513644e-06,9.224633e-05,3.613042e-05,2.262703e-05,2.667433e-05,8.131825e-05,7.939502e-05,8.233109e-05,...,3.347974e-05,1.044499e-04,1.326411e-05,4.485989e-05,4.822617e-05,0.000094,6.216209e-05,0.000303,9.330856e-05,3.069361e-05
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00
max,7.722008e-04,4.830918e-03,7.513148e-04,1.259446e-02,3.984064e-03,2.583979e-03,2.403846e-03,8.474576e-03,1.030928e-02,1.176471e-02,...,4.662005e-03,1.492537e-02,1.895375e-03,6.410256e-03,6.779661e-03,0.010638,8.882655e-03,0.037975,1.333333e-02,4.385965e-03


## Add gender labels

In [93]:
# Add gender column
gender_counts_df = counts 
gender_counts_df['gender'] = result_user['gender']
gender_counts_df = gender_counts_df.dropna()

In [94]:
# Preview counts with gender labels
gender_counts_df

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧙🏾,⛹🏼‍♀️,🏂🏾,🧜🏾‍♂️,🤸🏽,🦶🏾,🏋🏾,🇧🇹,🇳🇨,gender
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,male
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,male
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,male
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,female


In [97]:
# Count males and Females
gender_counts_df['gender']
males_count = 0
females_count = 0

for i in gender_counts_df['gender']:
    if i == 'female':
        females_count = females_count+1
    if i == 'male':
        males_count = males_count+1
print("Males: {}".format(males_count))
print("Females: {}".format(females_count))

Males: 5610
Females: 6138


In [98]:
# Female -> 0
# Male -> 1
gender_counts_df = gender_counts_df.replace('female', 0)
gender_counts_df = gender_counts_df.replace('male', 1)
gender_counts_df

,👨🏿‍🔬,🚵🏽‍♂️,👨🏾‍🎤,👩🏾‍🏫,🙍🏽‍♂️,🇮🇨,👳🏾‍♀️,👨🏾‍🏫,🦠,💂🏻‍♀️,...,🧙🏾,⛹🏼‍♀️,🏂🏾,🧜🏾‍♂️,🤸🏽,🦶🏾,🏋🏾,🇧🇹,🇳🇨,gender
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
17,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
19,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
21,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
29,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


## Split into test and train

In [99]:
X_train_gender, X_test_gender, y_train_gender, y_test_gender = train_test_split(gender_counts_df.drop(columns=['gender']), gender_counts_df['gender'], test_size=0.2, random_state=42)

## SVM with Chi-squared kernel Classification

In [82]:
svm_chi = SVC(kernel=chi2_kernel, probability=True).fit(X_train_gender, y_train_gender)

# Train Data
pred_prob_chi = svm_chi.predict_proba(X_train_gender)
logloss_train_chi = log_loss(y_train_gender, pred_prob_chi)
predict_train_chi  = svm_chi.predict(X_train_gender)
accuracy_train_chi = accuracy_score(y_train_gender, predict_train_chi)

# Test Data
pred_prob_val_chi = svm_chi.predict_proba(X_test_gender)
logloss_val_chi = log_loss(y_test_gender, pred_prob_val_chi)
predict_val_chi  = svm_chi.predict(X_test_gender)
accuracy_val_chi = accuracy_score(y_test_gender, predict_val_chi)

# Report
print('Log loss with SVM (Chi-squared kernel) on training set: {:.3f}'.format(logloss_train_chi))
print('Log loss with SVM (Chi-squared kernel) on test set: {:.3f}\n'.format(logloss_val_chi))
print('Accuracy score with SVM (Chi-squared kernel) on training set: {:.3f}'.format(accuracy_train_chi))
print('Accuracy score with SVM (Chi-squared kernel) on test set: {:.3f}'.format(accuracy_val_chi))

Log loss with SVM (Chi-squared kernel) on training set: 0.303
Log loss with SVM (Chi-squared kernel) on test set: 0.471

Accuracy score with SVM (Chi-squared kernel) on training set: 0.899
Accuracy score with SVM (Chi-squared kernel) on test set: 0.776


## Linear SVM Classification

In [100]:
clf = LinearSVC(random_state=0, tol=1e-5).fit(X_train_gender, y_train_gender)

# Train Data
predict_train_clf  = clf.predict(X_train_gender)
accuracy_train_clf = accuracy_score(y_train_gender, predict_train_clf)

# Test Data

predict_val_clf  = clf.predict(X_test_gender)
accuracy_val_clf = accuracy_score(y_test_gender, predict_val_clf)

# Report
print('Accuracy score with Linear SVM on training set: {:.3f}'.format(accuracy_train_clf))
print('Accuracy score with Linear SVM on test set: {:.3f}'.format(accuracy_val_clf))

Accuracy score with Linear SVM on training set: 0.774
Accuracy score with Linear SVM on test set: 0.751
